In [23]:
import pandas as pd
import requests
import lyricsgenius
from bs4 import BeautifulSoup
import time
import numpy as np
from random import  random
import os
import string
import pickle
from glob import glob

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius

Running List of Assumptions:
- Track is associated only with primary artist, not features (If there's an equally split collab we'll deal with that later). 
- Leaving out December bc Christmas (not sorry)
- All tracks carry equal weight regardless of duration, word count, etc

Need 2 data frames: 

1) Columns are the top 200 tracks for each week.
2) Each column is track, rows are the relevant data values.

In [15]:
weekly_200_df = pd.DataFrame(index = range(200))
for file in sorted(glob('data/regional-us-weekly-2022*.csv')):
    weekly_chart = pd.read_csv(file)
    weekly_200_df[file[-14:-4]] = weekly_chart['track_name']+'---'+weekly_chart['artist_names']
    

In [91]:
weekly_200_df.to_csv('data/weekly_200_df_2022.csv',index=False)

In [92]:
pd.read_csv('data/weekly_200_df_2022.csv')

,2022-01-06,2022-01-13,2022-01-20,2022-01-27,2022-02-03,2022-02-10,2022-02-17,2022-02-24,2022-03-03,2022-03-10,...,2022-09-22,2022-09-29,2022-10-06,2022-10-13,2022-10-20,2022-10-27,2022-11-03,2022-11-10,2022-11-17,2022-11-24
0,We Don't Talk About Bruno---Carolina Gaitán - ...,Sacrifice---The Weeknd,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,We Don't Talk About Bruno---Carolina Gaitán - ...,Heat Waves---Glass Animals,...,Bad Habit---Steve Lacy,"Unholy (feat. Kim Petras)---Sam Smith, Kim Petras","Unholy (feat. Kim Petras)---Sam Smith, Kim Petras","Unholy (feat. Kim Petras)---Sam Smith, Kim Petras","Unholy (feat. Kim Petras)---Sam Smith, Kim Petras",Anti-Hero---Taylor Swift,Anti-Hero---Taylor Swift,"Rich Flex---Drake, 21 Savage","Rich Flex---Drake, 21 Savage","Rich Flex---Drake, 21 Savage"
1,Heat Waves---Glass Animals,We Don't Talk About Bruno---Carolina Gaitán - ...,"pushin P (feat. Young Thug)---Gunna, Future, Y...","pushin P (feat. Young Thug)---Gunna, Future, Y...","pushin P (feat. Young Thug)---Gunna, Future, Y...","pushin P (feat. Young Thug)---Gunna, Future, Y...",Heat Waves---Glass Animals,Heat Waves---Glass Animals,Heat Waves---Glass Animals,We Don't Talk About Bruno---Carolina Gaitán - ...,...,"Me Porto Bonito---Bad Bunny, Chencho Corleone",Bad Habit---Steve Lacy,Bad Habit---Steve Lacy,Bad Habit---Steve Lacy,Bad Habit---Steve Lacy,Lavender Haze---Taylor Swift,Lavender Haze---Taylor Swift,"Major Distribution---Drake, 21 Savage",Anti-Hero---Taylor Swift,Anti-Hero---Taylor Swift
2,"INDUSTRY BABY (feat. Jack Harlow)---Lil Nas X,...",Gasoline---The Weeknd,Super Gremlin---Kodak Black,Super Gremlin---Kodak Black,Super Gremlin---Kodak Black,Super Gremlin---Kodak Black,"pushin P (feat. Young Thug)---Gunna, Future, Y...",Nail Tech---Jack Harlow,Super Gremlin---Kodak Black,Super Gremlin---Kodak Black,...,As It Was---Harry Styles,As It Was---Harry Styles,Tití Me Preguntó---Bad Bunny,Romantic Homicide---d4vd,Romantic Homicide---d4vd,Maroon---Taylor Swift,Midnight Rain---Taylor Swift,"Pussy & Millions (feat. Travis Scott)---Drake,...",Bad Habit---Steve Lacy,Bad Habit---Steve Lacy
3,"STAY (with Justin Bieber)---The Kid LAROI, Jus...",Out of Time---The Weeknd,Heat Waves---Glass Animals,"Surface Pressure - From ""Encanto""/Soundtrack V...","Surface Pressure - From ""Encanto""/Soundtrack V...",Heat Waves---Glass Animals,Super Gremlin---Kodak Black,"pushin P (feat. Young Thug)---Gunna, Future, Y...","pushin P (feat. Young Thug)---Gunna, Future, Y...","STAY (with Justin Bieber)---The Kid LAROI, Jus...",...,Tití Me Preguntó---Bad Bunny,"Me Porto Bonito---Bad Bunny, Chencho Corleone",As It Was---Harry Styles,As It Was---Harry Styles,Something in the Orange---Zach Bryan,Midnight Rain---Taylor Swift,Maroon---Taylor Swift,Anti-Hero---Taylor Swift,Midnight Rain---Taylor Swift,"Unholy (feat. Kim Petras)---Sam Smith, Kim Petras"
4,Super Gremlin---Kodak Black,Is There Someone Else?---The Weeknd,Surface Pressure---Jessica Darrow,Heat Waves---Glass Animals,Heat Waves---Glass Animals,Cigarettes---Juice WRLD,"Surface Pressure - From ""Encanto""/Soundtrack V...",Super Gremlin---Kodak Black,"STAY (with Justin Bieber)---The Kid LAROI, Jus...","pushin P (feat. Young Thug)---Gunna, Future, Y...",...,Romantic Homicide---d4vd,Romantic Homicide---d4vd,Romantic Homicide---d4vd,Tití Me Preguntó---Bad Bunny,As It Was---Harry Styles,Snow On The Beach (feat. Lana Del Rey)---Taylo...,Karma---Taylor Swift,"On BS---Drake, 21 Savage",Lavender Haze---Taylor Swift,Lavender Haze---Taylor Swift
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,thailand---Roddy Ricch,"Calling My Phone---Lil Tjay, 6LACK",Bring The Hook---YoungBoy Never Broke Again,"Back in Blood (feat. Lil Durk)---

In [19]:
all_tracks = weekly_200_df.stack().unique()

In [81]:
track = all_tracks[0]

In [123]:
with open('../assignment-03/spotify_creds.pkl', 'rb') as handle:
    spotify_creds = pickle.load(handle)
    
manager = SpotifyClientCredentials(
    client_id = spotify_creds['client_id'],
    client_secret = spotify_creds['client_secret']
)

sp = spotipy.Spotify(client_credentials_manager = manager)

    
with open('../assignment-03/genius_creds.pkl', 'rb') as handle:
    genius_creds = pickle.load(handle)
    
ACCESS_TOKEN = genius_creds['ACCESS_TOKEN'] 
API_BASE_URL = genius_creds['API_BASE_URL'] 
CLIENT_SECRET = genius_creds['CLIENT_SECRET'] 
REDIRECT_URI = genius_creds['REDIRECT_URI']

genius = lyricsgenius.Genius(ACCESS_TOKEN)

In [42]:
word_scores = pd.read_csv('../final_project/data/ousiometry_data_augmented.tsv', sep='\t', header=0)

In [83]:
track_search = sp.search(track.split(',')[0],type='track')
track_id = track_search['tracks']['items'][0]['id']
audio_features = sp.audio_features(track_id)
# audio_features = dict([(key,audio_features['track'][key]) for key in audio_features['track'].keys() if key not in ['codestring','echoprintstring','synchstring','rhythmstring']])


In [84]:
pd.Series(audio_features[0])

danceability                                                    0.633
energy                                                          0.871
key                                                                 4
loudness                                                       -7.067
mode                                                                0
speechiness                                                    0.0582
acousticness                                                    0.614
instrumentalness                                                    0
liveness                                                         0.11
valence                                                         0.906
tempo                                                         165.254
type                                                   audio_features
id                                             0Gx93PqcQlb5E0LIYvRfUD
uri                              spotify:track:0Gx93PqcQlb5E0LIYvRfUD
track_href          

In [98]:
# track_name = track.split('---')[0]
# artist_name = track.split('---')[1].split(',')[0].split(' - ')[0]
track_name="Ghost"
artist_name="Justin Bieber"
song_object = genius.search_song(title=track_name,artist=artist_name)

Searching for "Ghost" by Justin Bieber...
Done.


In [86]:
annotations = genius.song_annotations(song_id=song_object.id)


In [73]:
# I heard you like list comprehension
list_of_annotations = [a for la in [[item for sublist in entry[1] for item in sublist] for entry in annotations] for a in la]

In [76]:
track_annotations = '\n\n'.join(list_of_annotations)

In [77]:
ousio_df = pd.DataFrame(columns = word_scores.columns)
for word in track_annotations.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').replace('’',"'").lower().split(' '):
    ousio_df = pd.concat([ousio_df,word_scores[word_scores['word']==word]])

In [78]:
ousio_df

,word,valence,arousal,dominance,goodness,energy,structure,power,danger
16772,line,0.04098,-0.319966,-0.147155,0.012334,-0.350293,-0.053430,-0.238973,-0.256416
15592,is,0.16998,-0.259966,0.053845,0.211552,-0.212405,-0.097492,-0.000604,-0.299783
23916,reminiscent,-0.00002,-0.018966,0.110845,0.056176,0.044444,-0.086691,0.071149,-0.008295
29880,track,-0.05202,-0.066966,0.025845,-0.022487,-0.032942,-0.079169,-0.039194,-0.007392
26309,sings,0.35798,0.106034,-0.080155,0.254760,-0.013069,0.284155,0.170901,-0.189384
...,...,...,...,...,...,...,...,...,...
16894,living,0.36498,0.108034,0.226845,0.408298,0.153888,0.077144,0.397526,-0.179895
17342,make,0.18398,-0.079966,-0.020155,0.161139,-0.106247,0.058268,0.038815,-0.189070
25652,sense,0.28098,-0.082966,-0.026155,0.242476,-0.127446,0.107144,0.081339,-0.261575
21874,post,0.01998,-0.073966,0.047845,0.051334,-0.038275,-0.063711,0.009234,-0.063363


In [79]:
ousio_df[ousio_df.columns[1:]].mean()

valence      0.099869
arousal     -0.055914
dominance    0.019602
goodness     0.104041
energy      -0.051435
structure    0.003777
power        0.037199
danger      -0.109938
dtype: float64

In [88]:
song_data = pd.concat([pd.Series(audio_features[0]),ousio_df[ousio_df.columns[1:]].mean()])

In [119]:
song_data.index

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'valence', 'arousal', 'dominance', 'goodness',
       'energy', 'structure', 'power', 'danger'],
      dtype='object')

In [93]:
audio_features

[{'danceability': 0.633,
  'energy': 0.871,
  'key': 4,
  'loudness': -7.067,
  'mode': 0,
  'speechiness': 0.0582,
  'acousticness': 0.614,
  'instrumentalness': 0,
  'liveness': 0.11,
  'valence': 0.906,
  'tempo': 165.254,
  'type': 'audio_features',
  'id': '0Gx93PqcQlb5E0LIYvRfUD',
  'uri': 'spotify:track:0Gx93PqcQlb5E0LIYvRfUD',
  'track_href': 'https://api.spotify.com/v1/tracks/0Gx93PqcQlb5E0LIYvRfUD',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Gx93PqcQlb5E0LIYvRfUD',
  'duration_ms': 215360,
  'time_signature': 3}]

In [101]:
song_object._body['url']

'https://genius.com/Justin-bieber-ghost-lyrics'

In [105]:
response = requests.get('https://genius.com/Justin-bieber-ghost-lyrics')


In [108]:
import re
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(re.escape('SongDescription__Content'))
elements = soup.find_all(class_=pattern)


In [114]:
elements[0].text

'On “Ghost,” Justin Bieber sings about losing a loved one, whether from death or them walking out of his life. The song also accounts for how people could not see their families during the COVID-19 pandemic, in which areas enforced social distancing guidelines.\nIn a Tweet from February 20, 2022, Jon Bellion backgrounds the inspiration of the song:\nWrote this one about my grandmother. Really cool to feel her every time I turn the radio on in the car. Miss you Gma.\nOn February 26, 2021, Justin’s paternal grandfather, George, passed away unexpectedly in his house, the same day that Justin announced Justice. The cause of his death is unknown, but it could be a major factor in what Justin croons about in the song.'

In [115]:
len(word_scores)

32721

In [116]:
all_song_stats = pd.read_csv('data/track_data-2022.csv')

In [121]:
all_song_stats = pd.read_csv('data/track_data-2022.csv')
all_song_stats.index = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence_i', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'scored_words', 'valence_l', 'arousal', 'dominance', 'goodness',
       'energy', 'structure', 'power', 'danger']

In [122]:
all_song_stats

,"We Don't Talk About Bruno---Carolina Gaitán - La Gaita, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz, Encanto - Cast",Sacrifice---The Weeknd,Heat Waves---Glass Animals,As It Was---Harry Styles,First Class---Jack Harlow,N95---Kendrick Lamar,Running Up That Hill (A Deal With God)---Kate Bush,Glimpse of Us---Joji,Bad Habit---Steve Lacy,"Unholy (feat. Kim Petras)---Sam Smith, Kim Petras",...,Cabo---Koe Wetzel,Self Care---Mac Miller,Stir Fry---Migos,Son Of A Sinner---Jelly Roll,"To The Bone (feat. YoungBoy Never Broke Again)---Quavo, Takeoff, YoungBoy Never Broke Again",Shinunoga E-Wa---Fujii Kaze,Hayloft II---Mother Mother,"lovely (with Khalid)---Billie Eilish, Khalid",DNA.---Kendrick Lamar,Attention---NewJeans
danceability,0.577,0.735,0.761,0.52,0.902,0.79,0.679,0.44,0.686,0.714,...,0.64,0.52,0.817,0.365,0.899,0.6,0.326,0.351,0.638,0.811
energy,0.45,0.795,0.525,0.731,0.582,0.67,0.594,0.317,0.494,0.472,...,0.601,0.538,0.816,0.541,0.582,0.76,0.866,0.296,0.523,0.648
key,0,11,11,6,5,1,0,8,1,2,...,4,1,2,7,11,6,9,4,1,10
loudness,-8.516,-6.523,-6.9,-5.338,-5.902,-5.527,-11.135,-9.258,-7.093,-7.375,...,-7.172,-8.109,-5.402,-7.489,-7.275,-6.124,-4.541,-10.109,-6.664,-3.684
mode,0,0,1,0,0,1,1,1,1,1,...,0,1,1,1,0,0,0,0,1,0
speechiness,0.0834,0.113,0.0944,0.0557,0.109,0.105,0.108,0.0531,0.0355,0.0864,...,0.0372,0.206,0.269,0.0304,0.264,0.0452,0.101,0.0333,0.357,0.0368
acousticness,0.357,0.0296,0.44,0.342,0.111,0.377,0.0377,0.891,0.613,0.013,...,0.0172,0.367,0.00263,0.0419,0.0395,0.166,0.0044,0.934,0.00454,0.236
instrumentalness,0,3.18e-05,6.7e-06,0.00101,3.18e-06,2.32e-06,2.13e-05,4.78e-06,5.8e-05,4.51e-06,...,0,0.00187,0,2.25e-06,0,4.09e-05,7.37e-06,0,0,0
liveness,0.111,0.0678,0.0921,0.311,0.111,0.119,0.124,0.141,0.402,0.266,...,0.356,0.119,0.159,0.281,0.1,0.189,0.0912,0.095,0.0842,0.0761
valence_i,0.83,0.905,0.531,0.662,0.332,0.408,0.361,0.268,0.7,0.238,...,0.205,0.177,0.508,0.329,0.505,0.519,0.172,0.12,0.422,0.695


In [124]:
artist_search = sp.search("Kendrick Lamar",type='artist')
artist_search['artists']['items'][0]['genres']

['conscious hip hop', 'hip hop', 'rap', 'west coast rap']

In [125]:
artist_search = sp.search("Taylor Swift",type='artist')
artist_search['artists']['items'][0]['genres']

['pop']

In [126]:
artist_search = sp.search("Tyler, the Creator",type='artist')
artist_search['artists']['items'][0]['genres']

['hip hop', 'rap']

In [127]:
artist_search = sp.search("Tyler Childers",type='artist')
artist_search['artists']['items'][0]['genres']

['kentucky indie', 'kentucky roots', 'outlaw country']

In [128]:
artist_search = sp.search("Zedd",type='artist')
artist_search['artists']['items'][0]['genres']

[]

In [129]:
artist_means = pd.read_csv('data/artist_means.csv')

In [132]:
genres = []
for a in artist_means['artist']:
    artist_search = sp.search(a,type='artist')
    genres.append(artist_search['artists']['items'][0]['genres'])

In [133]:
genres

[['aesthetic rap', 'cloud rap', 'florida rap', 'underground hip hop'],
 ['cloud rap', 'dark trap', 'new orleans rap', 'underground hip hop'],
 ['atl hip hop', 'hip hop', 'rap'],
 ['cali rap', 'pop rap'],
 ['g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap'],
 ['movie tunes'],
 ['east coast hip hop',
  'gangster rap',
  'hip hop',
  'pop rap',
  'queens hip hop',
  'rap'],
 ['melodic rap', 'rap', 'trap'],
 ['east coast hip hop', 'hip hop', 'rap', 'trap'],
 ['pov: indie'],
 ['british soul', 'pop', 'pop soul', 'uk pop'],
 ['underground power pop'],
 ['garage rock', 'modern rock', 'permanent wave', 'rock', 'sheffield indie'],
 ['indie hip hop'],
 ['rap latina', 'trap queen'],
 ['k-pop', 'k-pop girl group', 'pop'],
 ['k-pop', 'k-pop boy group', 'pop'],
 ['hip hop', 'rap'],
 ['contemporary country', 'modern country pop'],
 ['art pop',
  'baltimore indie',
  'dream pop',
  'dreamo',
  'indie pop',
  'indie rock',
  'indietronica',
  'modern dream pop',
  'pov: indie'],
 ['pov: indie

In [134]:
genres_flat = [item for l in genres for item in l]

In [136]:
from collections import Counter
Counter(genres_flat)

Counter({'rap': 67,
         'pop': 54,
         'hip hop': 36,
         'trap': 31,
         'pov: indie': 31,
         'rock': 25,
         'pop rap': 22,
         'modern rock': 19,
         'melodic rap': 15,
         'atl hip hop': 13,
         'r&b': 12,
         'southern hip hop': 12,
         'gangster rap': 10,
         'permanent wave': 10,
         'classic rock': 9,
         'contemporary country': 8,
         'dance pop': 8,
         'soft rock': 8,
         'mellow gold': 7,
         'trap queen': 6,
         'album rock': 6,
         'hard rock': 6,
         'chicago rap': 6,
         'underground hip hop': 5,
         'west coast rap': 5,
         'movie tunes': 5,
         'uk pop': 5,
         'heartland rock': 5,
         'modern alternative rock': 5,
         'pluggnb': 5,
         'bedroom pop': 5,
         'new wave pop': 5,
         'cloud rap': 4,
         'florida rap': 4,
         'cali rap': 4,
         'g funk': 4,
         'east coast hip hop': 4,
        

In [138]:
[l for l in genres if (('hip hop' in l) and ('rap' not in l))]

[['alternative hip hop',
  'boom bap',
  'buffalo hip hop',
  'hip hop',
  'upstate ny rap'],
 ['g funk', 'gangster rap', 'hip hop', 'west coast rap']]

In [139]:
#any(substring in s for s in string_list)

In [146]:
genres_simplified = []
for l in genres:
    if (any('hip hop' in s for s in l) or (any('rap' in s for s in l))):
        genres_simplified.append('hip hop')
    elif any('country' in s for s in l):
        genres_simplified.append('country')
    elif any('rock' in s for s in l):
        genres_simplified.append('rock')
    elif any('pop' in s for s in l):
        genres_simplified.append('pop')
    
    elif any('r&b' in s for s in l):
        genres_simplified.append('r&b')
    elif any('indie' in s for s in l):
        genres_simplified.append('indie')
    elif (any('electronic' in s for s in l) or (any('edm' in s for s in l))):
        genres_simplified.append('electronic')
    elif any('jazz' in s for s in l):
        genres_simplified.append('jazz')
    elif any('reggae' in s for s in l):
        genres_simplified.append('reggae')
    elif any('latin' in s for s in l): #using this generalization sucks
        genres_simplified.append('latin')
    elif any('classical' in s for s in l):
        genres_simplified.append('classical') #sure why not
    else:
        genres_simplified.append('niche or unknown')
    
    
    
    
    
    

In [147]:
len(genres_simplified)

281

In [148]:
len(genres)

281

In [149]:
artist_means['genre'] = genres_simplified

In [150]:
artist_means[['artist','genre']]

,artist,genre
0,$NOT,hip hop
1,$uicideboy$,hip hop
2,21 Savage,hip hop
3,24kGoldn,hip hop
4,2Pac,hip hop
...,...,...
276,blink-182,rock
277,d4vd,pop
278,j-hope,hip hop
279,lil Shordie Scott,hip hop


In [151]:
artist_means.to_csv('data/artist_means.csv',index=False)

In [153]:
artist_genre_dict = dict(zip(artist_means['artist'].tolist(),artist_means['genre'].tolist()))
song_stats['genre'] = song_stats['artist'].map(artist_genre_dict)

NameError: name 'song_stats' is not defined